# Test04: test code for Broadcast

# Preparation 

## Preparation of google drive (if required)

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch211217/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211217/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [ ]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

## C++コードをコンパイル、実行

In [9]:
! cat ./test/test04.cpp


#include "minictorch.hpp"

void do_broadcast( Tensor &a, Tensor &b )
{
    VariableTensor va(a);
    VariableTensor vb(b);
    
    ListConstructOp  list1;
    list1.set_inputs( &va );
    list1.set_inputs( &vb );
    
    BroadcastTensorsOp  bcast1;
    bcast1.set_inputs( &list1 );
    
    ListUnpackOp unpk0( 0 );
    unpk0.set_inputs( &bcast1 );
    ListUnpackOp unpk1( 1 );
    unpk1.set_inputs( &bcast1 );
    
    AddOp add1;
    add1.set_inputs( &unpk0 );
    add1.set_inputs( &unpk1 );
    add1.set_inputs( NULL );
    
    cout<<"--------------------------------"<<endl;
    cout<<"a"<<a<<endl;
    cout<<"b"<<b<<endl;
    
    list1.forward();
    if( bcast1.forward() )
    {
        unpk0.forward();
        unpk1.forward();
        add1.forward();
        
        cout<<"a_mod"<<unpk0.output<<endl;
        cout<<"b_mod"<<unpk1.output<<endl;
        cout<<"a+b"<<add1.output<<endl;
    }
}

void _broadcast()
{
    Tensor a  = xt::zeros<fprec>({3, 3});
    Tensor b  = xt::arange(3);

In [7]:
C_SRC = './test/test04.cpp'
C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include '
C_OUT = './test/test04'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04


In [8]:
! ./test/test04

--------------------------------
a{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b{ 0.,  1.,  2.}
broadcast shape 3,3,
broadcast check=1
a_mod{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b_mod{{ 0.,  1.,  2.},
 { 0.,  1.,  2.},
 { 0.,  1.,  2.}}
a+b{{ 0.,  1.,  2.},
 { 0.,  1.,  2.},
 { 0.,  1.,  2.}}
--------------------------------
a{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b{{ 0.},
 { 1.},
 { 2.}}
broadcast shape 3,3,
broadcast check=1
a_mod{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b_mod{{ 0.,  0.,  0.},
 { 1.,  1.,  1.},
 { 2.,  2.,  2.}}
a+b{{ 0.,  0.,  0.},
 { 1.,  1.,  1.},
 { 2.,  2.,  2.}}
--------------------------------
a{ 0.,  1.,  2.,  3.}
b{{ 0.},
 { 1.},
 { 2.}}
broadcast shape 3,4,
broadcast check=1
a_mod{{ 0.,  1.,  2.,  3.},
 { 0.,  1.,  2.,  3.},
 { 0.,  1.,  2.,  3.}}
b_mod{{ 0.,  0.,  0.,  0.},
 { 1.,  1.,  1.,  1.},
 { 2.,  2.,  2.,  2.}}
a+b{{ 0.,  1.,  2.,  3.},
 { 1.,  2.,  3.,  4.},
 { 2.,  3.,  4.,  5.}}
---------------

## Test Broadcast (python)

In [ ]:
import numpy as np
a = np.zeros((3, 3), np.int)
print(a)
b = np.arange(3)
print(b)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
[0 1 2]


In [ ]:
np.broadcast_to(b,(3,3))

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [ ]:
c=b.reshape(3,1)
print(c)

[[0]
 [1]
 [2]]


In [ ]:
np.broadcast_to(c,(3,3))

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]])

In [ ]:
a+b

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [ ]:
a+c

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]])

In [ ]:
b+c

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [ ]:
d = np.arange(4)
print(d)

[0 1 2 3]


In [ ]:
d+c

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5]])

In [ ]:
np.broadcast_to(c,(3,4))

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2]])

In [ ]:
np.broadcast_to(d,(3,4))

array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

In [ ]:
x = np.zeros((4, 3), dtype=np.int)
print(x)

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [ ]:
y = np.arange(6).reshape(2, 3)
print(y)

[[0 1 2]
 [3 4 5]]


In [ ]:
x+y

ValueError: ignored

In [ ]:
try:
  print(x+y)
except:
  print("broadcast error. shapes", a.shape, ",",  b.shape)

broadcast error. shapes (3, 3) , (3,)


In [ ]:
def check_broadcast( a, b ):

  import numbers
  def is_scalar(a):
    return isinstance( a, numbers.Number )

  def check_shape( a, n_dim ):
    na = np.ones( n_dim, dtype=np.int )
    inc = n_dim - a.ndim
    for i in range(a.ndim):
      na[i+inc] = a.shape[i]
    return na

  ns = 0
  if is_scalar(a):  ns += 1
  if is_scalar(b):  ns += 1
  if( ns > 0 ): 
    print("scalar =",ns)
    return 2

  n_dim = max( a.ndim, b.ndim )
  na = check_shape( a, n_dim )
  nb = check_shape( b, n_dim )
  print("dimension", n_dim,na,nb)

  ne = 0
  n1 = 0
  n2 = 0
  for i in range(n_dim):
    if na[i] == 1:  
      n1 += 1
    elif nb[i] == 1:  
      n1 += 1
    elif na[i] == nb[i]:
      ne += 1
    else:
      #print("broadcast mismatch dimension no.",i)
      n2 += 1
  print("equal=",ne," eq1 =",n1," other =",n2)
  status = 1;
  if( ne == n_dim ):   
    pass
  elif ( (ne+n1) == n_dim ):  
    pass
  else:
    print("broadcast error. shapes=", a.shape, ",",  b.shape)
    status = 0
  print("status =",status)
  return status

In [ ]:
check_broadcast(a,b)

dimension 2 [3 3] [1 3]
equal= 1  eq1 = 1  other = 0
status = 1


1

In [ ]:
check_broadcast(a,c)

dimension 2 [3 3] [3 1]
equal= 1  eq1 = 1  other = 0
status = 1


1

In [ ]:
check_broadcast(d,c)

dimension 2 [1 4] [3 1]
equal= 0  eq1 = 2  other = 0
status = 1


1

In [ ]:
check_broadcast(x,y)

dimension 2 [4 3] [2 3]
equal= 1  eq1 = 0  other = 1
broadcast error. shapes= (4, 3) , (2, 3)
status = 0


0

In [ ]:
e=1
check_broadcast(a,e)

scalar = 1


2

In [ ]:
a+e

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])